# Import packages:

In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd

# Load maps:

In [2]:
maps_data_dir = Path("../data/raw/maps/")

In [3]:
spain_map_communities_level = gpd.read_file(
    maps_data_dir
    / "lineas_limite"
    / "SHP_ETRS89"
    / "recintos_autonomicas_inspire_peninbal_etrs89"
    / "recintos_autonomicas_inspire_peninbal_etrs89.shp"
)

In [4]:
spain_map_provinces_level = gpd.read_file(
    maps_data_dir
    / "lineas_limite"
    / "SHP_ETRS89"
    / "recintos_provinciales_inspire_peninbal_etrs89"
    / "recintos_provinciales_inspire_peninbal_etrs89.shp"
)

In [5]:
canary_islands_map_communities_level = gpd.read_file(
    maps_data_dir
    / "lineas_limite"
    / "SHP_REGCAN95"
    / "recintos_autonomicas_inspire_canarias_regcan95"
    / "recintos_autonomicas_inspire_canarias_regcan95.shp"
)

In [6]:
canary_islands_map_provinces_level = gpd.read_file(
    maps_data_dir
    / "lineas_limite"
    / "SHP_REGCAN95"
    / "recintos_provinciales_inspire_canarias_regcan95"
    / "recintos_provinciales_inspire_canarias_regcan95.shp"
)

In [7]:
df_1 = (
    spain_map_provinces_level.loc[:, ["NAMEUNIT", "CODNUT2", "geometry"]]
    .merge(
        right=spain_map_communities_level.loc[:, ["NAMEUNIT", "CODNUT1", "CODNUT2"]],
        left_on="CODNUT2",
        right_on="CODNUT2",
        suffixes=("_province", "_community"),
    )
    .loc[:, ["NAMEUNIT_community", "NAMEUNIT_province", "geometry"]]
    .rename(
        columns={"NAMEUNIT_community": "community", "NAMEUNIT_province": "province"}
    )
)

In [8]:
df_2 = (
    canary_islands_map_provinces_level.loc[:, ["NAMEUNIT", "CODNUT2", "geometry"]]
    .merge(
        right=canary_islands_map_communities_level.loc[
            :, ["NAMEUNIT", "CODNUT1", "CODNUT2"]
        ],
        left_on="CODNUT2",
        right_on="CODNUT2",
        suffixes=("_province", "_community"),
    )
    .loc[:, ["NAMEUNIT_community", "NAMEUNIT_province", "geometry"]]
    .rename(
        columns={"NAMEUNIT_community": "community", "NAMEUNIT_province": "province"}
    )
)

In [9]:
df_2 = df_2.to_crs(df_1.crs)

In [10]:
final_map = pd.concat([df_1, df_2], ignore_index=True)

In [11]:
final_map.head()

,community,province,geometry
0,País Vasco/Euskadi,Araba/Álava,"POLYGON ((-2.76808 42.61408, -2.76863 42.61334..."
1,Castilla-La Mancha,Albacete,"POLYGON ((-2.55212 38.08501, -2.55207 38.08517..."
2,Comunitat Valenciana,Alacant/Alicante,"MULTIPOLYGON (((-0.75223 37.88691, -0.75225 37..."
3,Andalucía,Almería,"MULTIPOLYGON (((-3.03624 35.93791, -3.03637 35..."
4,Castilla y León,Ávila,"POLYGON ((-5.43382 40.24491, -5.4343 40.24276,..."


# Remove `Gibraltar` and `Pheasant Island`:

In [12]:
final_map = final_map.query(
    "province not in ['Gibraltar. Territorio no asociado a ninguna provincia', 'Territorios no asociados a ninguna provincia']"
)

In [13]:
final_map.head()

,community,province,geometry
0,País Vasco/Euskadi,Araba/Álava,"POLYGON ((-2.76808 42.61408, -2.76863 42.61334..."
1,Castilla-La Mancha,Albacete,"POLYGON ((-2.55212 38.08501, -2.55207 38.08517..."
2,Comunitat Valenciana,Alacant/Alicante,"MULTIPOLYGON (((-0.75223 37.88691, -0.75225 37..."
3,Andalucía,Almería,"MULTIPOLYGON (((-3.03624 35.93791, -3.03637 35..."
4,Castilla y León,Ávila,"POLYGON ((-5.43382 40.24491, -5.4343 40.24276,..."


In [14]:
output_path = Path("../data/processed/maps/")

In [15]:
final_map.to_file(output_path / "spain_map.shp", index=False)